# Logistic Regression example

In [2]:
// cargo install the newest version
// this may take a while, needs to download and install the library
:dep smartcore = { git = "https://github.com/smartcorelib/smartcore", branch = "v0.5-wip" }

In [3]:
use smartcore::dataset::iris::load_dataset as iris_load;
use smartcore::dataset::breast_cancer::load_dataset as bc_load;
use smartcore::dataset::diabetes::load_dataset as db_load;
use smartcore::linear::logistic_regression::LogisticRegression;
use smartcore::linalg::basic::matrix::DenseMatrix;
// Model performance
use smartcore::metrics::accuracy;

use smartcore::model_selection::{cross_val_predict, cross_validate, KFold};

## basic prediction and accuracy

In [4]:
// Load Iris dataset
let iris_dataset = iris_load();

// Turn Iris dataset into NxM matrix
// Input data
let x: DenseMatrix<f32> = DenseMatrix::new(
    iris_dataset.num_samples,      // num rows
    iris_dataset.num_features,     // num columns
    iris_dataset.data,             // data as Vec
    false,                         // column_major
);
// These are our target class labels
let y: Vec<u32> = iris_dataset.target;

// Fit Logistic Regression to Iris dataset
let lr = LogisticRegression::fit(&x, &y, Default::default()).unwrap();
let y_hat = lr.predict(&x).unwrap(); // Predict class labels

// Calculate training error
println!("accuracy: {}", accuracy(&y, &y_hat)); // Prints 0.98

accuracy: 0.9866666666666667


## basic cross validation

In [8]:
// we need to import this as LogisticRegression is a SupervisedEstimator
// and we need its traits
use smartcore::api::SupervisedEstimator

// This example is expired by
// https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_predict.html
// Load dataset
let breast_cancer_data = bc_load();
let x: DenseMatrix<f32> = DenseMatrix::new(
    breast_cancer_data.num_samples,
    breast_cancer_data.num_features,
    breast_cancer_data.data,
    false
);
// These are our target values
let y = breast_cancer_data.target;
// cross-validated estimator
let results = cross_validate(
    &LogisticRegression::new(),
    &x,
    &y,
    Default::default(),
    &KFold::default().with_n_splits(3),
    &accuracy,
)
.unwrap();
println!(
    "Test score: {}, training score: {}",
    results.mean_test_score(),
    results.mean_train_score()
);


Test score: 0.9507658033973824, training score: 0.9692450122668147


## basic cross validation with prediction

In [22]:
use smartcore::linear::linear_regression::LinearRegression;

// Load Diabetes dataset
let diabetes_data = db_load();
let x = DenseMatrix::new(
    diabetes_data.num_samples,
    diabetes_data.num_features,
    diabetes_data.data,
    false
);

// These are our target values
let y = diabetes_data.target;

// Generate cross-validated estimates for each input data point
let y_hat = cross_val_predict(
    &LinearRegression::new(),
    &x,
    &y,
    Default::default(),
    &KFold::default().with_n_splits(10),
)
.unwrap();


println!(
    "y hat {:?}",
    &y_hat
);


y hat [205, 70, 176, 169, 126, 104, 74, 126, 157, 210, 99, 104, 115, 164, 107, 168, 211, 183, 149, 125, 121, 90, 113, 243, 165, 148, 92, 179, 136, 175, 157, 71, 253, 114, 78, 90, 205, 154, 240, 140, 152, 73, 144, 78, 220, 124, 144, 109, 73, 189, 157, 167, 136, 159, 135, 69, 208, 83, 96, 134, 116, 185, 60, 104, 115, 192, 146, 124, 113, 122, 74, 237, 136, 126, 152, 129, 193, 69, 167, 92, 178, 122, 59, 152, 56, 169, 44, 157, 85, 104, 74, 185, 197, 59, 104, 126, 207, 213, 125, 139, 168, 108, 146, 153, 158, 117, 66, 155, 224, 144, 30, 123, 148, 207, 292, 187, 217, 228, 164, 147, 156, 198, 216, 177, 169, 190, 58, 110, 94, 205, 246, 70, 115, 70, 139, 236, 59, 238, 253, 250, 154, 221, 165, 117, 178, 243, 192, 227, 112, 179, 206, 145, 196, 123, 150, 199, 145, 122, 85, 237, 82, 227, 145, 202, 142, 77, 56, 264, 227, 222, 41, 87, 222, 95, 167, 122, 154, 222, 97, 166, 176, 90, 173, 157, 199, 186, 199, 64, 154, 115, 197, 127, 97, 135, 154, 168, 97, 187, 141, 178, 92, 70, 157, 197, 173, 228, 158, 211

In [33]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }

use plotters::prelude::*;

//Assemble XY dataset for the scatter plot
let xy: Vec<Vec<u32>> = y.clone()
        .into_iter()
        .zip(y_hat.clone().into_iter())
        .map(|(x1, x2)| vec![x1, x2])
        .collect();

let diff: Vec<Vec<u32>> = y.clone()
        .into_iter()
        .zip(y_hat.clone().into_iter())
        .map(|(x1, x2)| vec![(u32::max(x1, x2) - u32::min(x1, x2)), x2])
        .collect();

// println!("{:?}",
//     &xy
// );

evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Y and Y_hat (green) and their difference (blue)", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0u32..350u32, 0u32..400u32)?;
    
    chart.configure_mesh()
        .x_desc("Y")
        .y_desc("Y_hat")
        .draw()?;
    
    chart.draw_series(xy.iter().map(|x| Circle::new((x[0], x[1]), 3, GREEN.filled())))?;
    
    chart.draw_series(diff.iter().map(|x| Circle::new((x[0], x[1]), 3, BLUE.filled())))?;
    
    // You can alawys freely draw on the drawing backend
    let area = chart.plotting_area();
    let sd = 0.13;
    let two_sigma = sd * 2.0;
    area.draw(&Rectangle::new(
        [((0.5 - two_sigma) as u32, (0.5 - two_sigma)  as u32),
            ((0.5 + two_sigma)  as u32, (0.5 + two_sigma)  as u32)], 
        RED.mix(0.3).filled())
    )?;
    
    Ok(())
}).style("width: 60%")

Y and Y_hat (green) and their difference (blue)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Y_hat
 
 
Y
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
350
 
 
 
400
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
350